# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa08a332430>
├── 'a' --> tensor([[ 2.5595,  0.3434, -1.3231],
│                   [ 0.2859, -2.1064,  1.0352]])
└── 'x' --> <FastTreeValue 0x7fa08a332370>
    └── 'c' --> tensor([[-0.8252, -0.1941, -1.3498,  1.6579],
                        [ 1.3463,  0.4861,  1.6242,  1.3401],
                        [-0.1995, -0.9822,  0.6854, -0.7431]])

In [4]:
t.a

tensor([[ 2.5595,  0.3434, -1.3231],
        [ 0.2859, -2.1064,  1.0352]])

In [5]:
%timeit t.a

76.1 ns ± 1.23 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa08a332430>
├── 'a' --> tensor([[-2.1863, -0.2561, -1.6386],
│                   [-0.2053,  1.3676, -1.0387]])
└── 'x' --> <FastTreeValue 0x7fa08a332370>
    └── 'c' --> tensor([[-0.8252, -0.1941, -1.3498,  1.6579],
                        [ 1.3463,  0.4861,  1.6242,  1.3401],
                        [-0.1995, -0.9822,  0.6854, -0.7431]])

In [7]:
%timeit t.a = new_value

81.7 ns ± 0.954 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.5595,  0.3434, -1.3231],
               [ 0.2859, -2.1064,  1.0352]]),
    x: Batch(
           c: tensor([[-0.8252, -0.1941, -1.3498,  1.6579],
                      [ 1.3463,  0.4861,  1.6242,  1.3401],
                      [-0.1995, -0.9822,  0.6854, -0.7431]]),
       ),
)

In [10]:
b.a

tensor([[ 2.5595,  0.3434, -1.3231],
        [ 0.2859, -2.1064,  1.0352]])

In [11]:
%timeit b.a

73.4 ns ± 1.49 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.2500,  0.8521,  1.7052],
               [-0.8991,  0.1744,  0.6466]]),
    x: Batch(
           c: tensor([[-0.8252, -0.1941, -1.3498,  1.6579],
                      [ 1.3463,  0.4861,  1.6242,  1.3401],
                      [-0.1995, -0.9822,  0.6854, -0.7431]]),
       ),
)

In [13]:
%timeit b.a = new_value

656 ns ± 4.58 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 20.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.4 µs ± 144 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

301 µs ± 9.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

280 µs ± 13.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9fd054dbb0>
├── 'a' --> tensor([[[ 2.5595,  0.3434, -1.3231],
│                    [ 0.2859, -2.1064,  1.0352]],
│           
│                   [[ 2.5595,  0.3434, -1.3231],
│                    [ 0.2859, -2.1064,  1.0352]],
│           
│                   [[ 2.5595,  0.3434, -1.3231],
│                    [ 0.2859, -2.1064,  1.0352]],
│           
│                   [[ 2.5595,  0.3434, -1.3231],
│                    [ 0.2859, -2.1064,  1.0352]],
│           
│                   [[ 2.5595,  0.3434, -1.3231],
│                    [ 0.2859, -2.1064,  1.0352]],
│           
│                   [[ 2.5595,  0.3434, -1.3231],
│                    [ 0.2859, -2.1064,  1.0352]],
│           
│                   [[ 2.5595,  0.3434, -1.3231],
│                    [ 0.2859, -2.1064,  1.0352]],
│           
│                   [[ 2.5595,  0.3434, -1.3231],
│                    [ 0.2859, -2.1064,  1.0352]]])
└── 'x' --> <FastTreeValue 0x7f9fd054d760>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

47.1 µs ± 763 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa05c960250>
├── 'a' --> tensor([[ 2.5595,  0.3434, -1.3231],
│                   [ 0.2859, -2.1064,  1.0352],
│                   [ 2.5595,  0.3434, -1.3231],
│                   [ 0.2859, -2.1064,  1.0352],
│                   [ 2.5595,  0.3434, -1.3231],
│                   [ 0.2859, -2.1064,  1.0352],
│                   [ 2.5595,  0.3434, -1.3231],
│                   [ 0.2859, -2.1064,  1.0352],
│                   [ 2.5595,  0.3434, -1.3231],
│                   [ 0.2859, -2.1064,  1.0352],
│                   [ 2.5595,  0.3434, -1.3231],
│                   [ 0.2859, -2.1064,  1.0352],
│                   [ 2.5595,  0.3434, -1.3231],
│                   [ 0.2859, -2.1064,  1.0352],
│                   [ 2.5595,  0.3434, -1.3231],
│                   [ 0.2859, -2.1064,  1.0352]])
└── 'x' --> <FastTreeValue 0x7f9fd054dca0>
    └── 'c' --> tensor([[-0.8252, -0.1941, -1.3498,  1.6579],
                        [ 1.3463,  0.4861,  1.6242,  1.3401],
                 

In [23]:
%timeit t_cat(trees)

43.7 µs ± 603 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

86.6 µs ± 477 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.8252, -0.1941, -1.3498,  1.6579],
                       [ 1.3463,  0.4861,  1.6242,  1.3401],
                       [-0.1995, -0.9822,  0.6854, -0.7431]],
              
                      [[-0.8252, -0.1941, -1.3498,  1.6579],
                       [ 1.3463,  0.4861,  1.6242,  1.3401],
                       [-0.1995, -0.9822,  0.6854, -0.7431]],
              
                      [[-0.8252, -0.1941, -1.3498,  1.6579],
                       [ 1.3463,  0.4861,  1.6242,  1.3401],
                       [-0.1995, -0.9822,  0.6854, -0.7431]],
              
                      [[-0.8252, -0.1941, -1.3498,  1.6579],
                       [ 1.3463,  0.4861,  1.6242,  1.3401],
                       [-0.1995, -0.9822,  0.6854, -0.7431]],
              
                      [[-0.8252, -0.1941, -1.3498,  1.6579],
                       [ 1.3463,  0.4861,  1.6242,  1.3401],
                       [-0.1995, -0.9822,  0.6854, -0.7431]],

In [26]:
%timeit Batch.stack(batches)

110 µs ± 1.81 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.8252, -0.1941, -1.3498,  1.6579],
                      [ 1.3463,  0.4861,  1.6242,  1.3401],
                      [-0.1995, -0.9822,  0.6854, -0.7431],
                      [-0.8252, -0.1941, -1.3498,  1.6579],
                      [ 1.3463,  0.4861,  1.6242,  1.3401],
                      [-0.1995, -0.9822,  0.6854, -0.7431],
                      [-0.8252, -0.1941, -1.3498,  1.6579],
                      [ 1.3463,  0.4861,  1.6242,  1.3401],
                      [-0.1995, -0.9822,  0.6854, -0.7431],
                      [-0.8252, -0.1941, -1.3498,  1.6579],
                      [ 1.3463,  0.4861,  1.6242,  1.3401],
                      [-0.1995, -0.9822,  0.6854, -0.7431],
                      [-0.8252, -0.1941, -1.3498,  1.6579],
                      [ 1.3463,  0.4861,  1.6242,  1.3401],
                      [-0.1995, -0.9822,  0.6854, -0.7431],
                      [-0.8252, -0.1941, -1.3498,  1.6579],
                   

In [28]:
%timeit Batch.cat(batches)

201 µs ± 2.48 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

630 µs ± 12 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
